In [23]:
from datetime import datetime
from sqlalchemy import create_engine
import pymysql

In [1]:

import mysql.connector
import pandas as pd

def calcular_roi(simbolo, fecha_inicio, fecha_fin):
    # Conexión a la base de datos
    db_connection = mysql.connector.connect(
        host="127.0.0.1",
        user="root",  # Cambia por tu usuario de MySQL
        password="11jablum11",  # Cambia por tu contraseña de MySQL
        database="yfinance"
    )
    cursor = db_connection.cursor()



    # Consulta SQL para obtener el precio inicial y final en el periodo
    query = """
    SELECT fecha, precio_cierre
    FROM precios_historicos
    WHERE id_empresa = (SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s)
      AND fecha BETWEEN %s AND %s
    ORDER BY fecha ASC;
    """
    
    cursor.execute(query, (simbolo, fecha_inicio, fecha_fin))
    precios = cursor.fetchall()
    
    # Cerrar conexión
    cursor.close()
    db_connection.close()

    # Verifica si se obtuvieron precios en el rango de fechas
    if len(precios) < 2:
        st.warning("No se encontraron datos suficientes en el rango de fechas seleccionado.")
        return None

    # Precio inicial y final
    precio_inicial = precios[0][1]
    precio_final = precios[-1][1]
    
    # Calcular ROI
    roi = (precio_final - precio_inicial) / precio_inicial * 100

    return roi, pd.DataFrame(precios, columns=['Fecha', 'Precio Cierre'])

### Paso 2: Crear la Interfaz de Streamlit



In [29]:
import mysql.connector
import pandas as pd
import streamlit as st

def calcular_roi(simbolo, fecha_inicio, fecha_fin):
    connection = None
    cursor = None
    try:
        # Conexión a la base de datos
        host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'  # Cambia por tu endpoint de RDS
        user = 'admin'                       # Cambia por tu usuario de RDS
        password = '11jablum11'                   # Cambia por tu contraseña de RDS
        database = 'yfinance'              # Cambia por el nombre de tu base de datos
        port = 3306   

        connection = mysql.connector.connect(host=host, user=user, password=password, database=database, port=port)
        cursor = connection.cursor()

        # Consulta SQL para obtener el precio inicial y final en el periodo
        query = """
        SELECT fecha, precio_cierre
        FROM precios_historicos
        WHERE id_empresa = (SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s)
          AND fecha BETWEEN %s AND %s
        ORDER BY fecha ASC;
        """
        
        cursor.execute(query, (simbolo, fecha_inicio, fecha_fin))
        precios = cursor.fetchall()
        
    except mysql.connector.Error as err:
        st.error(f"Error al conectar a la base de datos: {err}")
        return None, None
    finally:
        # Cerrar cursor y conexión si fueron creados
        if cursor:
            cursor.close()
        if connection:
            connection.close()

    # Verifica si se obtuvieron precios en el rango de fechas
    if len(precios) < 2:
        st.warning("No se encontraron datos suficientes en el rango de fechas seleccionado.")
        return None, None

    # Precio inicial y final
    precio_inicial = precios[0][1]
    precio_final = precios[-1][1]
    
    # Calcular ROI
    roi = (precio_final - precio_inicial) / precio_inicial * 100

    return roi, pd.DataFrame(precios, columns=['Fecha', 'Precio Cierre'])




In [30]:
simbolo = "MMM"  # El símbolo de la acción a consultar
fecha_inicio = datetime(2000, 1, 1)  # Fecha de inicio del periodo
fecha_fin = datetime(2023, 12, 31)   # Fecha de fin del periodo

# Llamar a la función y capturar los resultados
roi, precios_df = calcular_roi(simbolo, fecha_inicio, fecha_fin)

# Verificar y mostrar el resultado
if roi is not None:
    print(f"ROI Total para {simbolo} desde {fecha_inicio.date()} hasta {fecha_fin.date()}: {roi:.2f}%")
    print("\nPrecios en el periodo:")
    print(precios_df)
else:
    print("No se encontraron datos suficientes para calcular el ROI en el rango de fechas seleccionado.")


2024-11-12 10:42:04.224 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-12 10:42:04.286 
  command:

    streamlit run /Users/javier/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-11-12 10:42:04.287 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


No se encontraron datos suficientes para calcular el ROI en el rango de fechas seleccionado.


In [37]:
import pymysql
import pandas as pd
from datetime import datetime

def calcular_roi(simbolo, fecha_inicio, fecha_fin):
    connection = None
    cursor = None
    try:
        # Configuración de la conexión a la base de datos
        host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'  # Cambia por tu endpoint de RDS
        user = 'admin'                       # Cambia por tu usuario de RDS
        password = '11jablum11'              # Cambia por tu contraseña de RDS
        database = 'yfinance'                # Cambia por el nombre de tu base de datos
        port = 3306   

        # Conectar a la base de datos
        connection = pymysql.connect(host=host, user=user, password=password, database=database, port=port)
        cursor = connection.cursor()

        # Paso 1: Obtener el id_empresa basado en el símbolo de la empresa
        cursor.execute("SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s", (simbolo,))
        resultado = cursor.fetchone()
        if not resultado:
            print("No se encontró la empresa en la base de datos.")
            return None, None
        id_empresa = resultado[0]

        # Paso 2: Obtener los precios de cierre en el rango de fechas especificado
        query = """
        SELECT fecha, precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha BETWEEN %s AND %s
        ORDER BY fecha ASC;
        """
        cursor.execute(query, (id_empresa, fecha_inicio, fecha_fin))
        precios = cursor.fetchall()

        # Si no hay precios en el rango, obtener el primer y último precio disponibles
        if len(precios) < 2:
            print("No se encontraron datos suficientes en el rango seleccionado. Usando datos completos.")
            query_all = """
            SELECT fecha, precio_cierre
            FROM precios_historicos
            WHERE id_empresa = %s
            ORDER BY fecha ASC;
            """
            cursor.execute(query_all, (id_empresa,))
            precios = cursor.fetchall()
            
    except pymysql.Error as err:
        print(f"Error al conectar a la base de datos o ejecutar la consulta: {err}")
        return None, None
    finally:
        # Cerrar cursor y conexión si fueron creados
        if cursor:
            cursor.close()
        if connection:
            connection.close()

    # Verifica si se obtuvieron precios después de intentar el rango completo
    if len(precios) < 2:
        print("No se encontraron datos suficientes para calcular el ROI.")
        return None, None

    # Precio inicial y final
    precio_inicial = precios[0][1]
    precio_final = precios[-1][1]
    
    # Calcular ROI
    roi = (precio_final - precio_inicial) / precio_inicial * 100

    return roi, pd.DataFrame(precios, columns=['Fecha', 'Precio Cierre'])

# Parámetros de ejemplo
simbolo = "APPL"  # El símbolo de la acción a consultar
fecha_inicio = datetime(2000, 1, 1)  # Fecha de inicio del periodo
fecha_fin = datetime(2023, 12, 31)   # Fecha de fin del periodo

# Llamar a la función y capturar los resultados
roi, precios_df = calcular_roi(simbolo, fecha_inicio, fecha_fin)

# Verificar y mostrar el resultado
if roi is not None:
    print(f"ROI Total para {simbolo} desde {fecha_inicio.date()} hasta {fecha_fin.date()}: {roi:.2f}%")
    print("\nPrecios en el periodo:")
    print(precios_df)
else:
    print("No se encontraron datos suficientes para calcular el ROI.")



No se encontró la empresa en la base de datos.
No se encontraron datos suficientes para calcular el ROI.
